In [9]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math


mp_draining = mp.solutions.drawing_utils # drawing 
mp_pose = mp.solutions.pose # import pose estimation model

## Calculate stuff

In [2]:
import numpy as np
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] -b[1], c[0]-b[0]) - np.arctan2(a[1] -b[1], a[0] - b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [3]:
import numpy as np
def calculate_distance(point1, point2):
    distance = ((point1[0] - point2[0])**2 + (point1[1]-point2[1])**2)**(0.5)
    return distance

In [4]:
def bentFinger(finger):
    if (calculate_angle(finger[0], finger[1], finger[3]) <90 and finger[3][1] > finger[0][1]):
        return True
    return False


In [5]:
def straightFinger(finger):
    if (finger[0][1] > finger[1][1] and finger[1][1] > finger[2][1] and finger[2][1] > finger[3][1]):
        return True
    return False

In [6]:
def straightDown(finger):
    if (finger[0][1] < finger[1][1] and finger[1][1] < finger[2][1] and finger[2][1] < finger[3][1]):
        return True
    return False

In [7]:
def isLeft(finger):
    if(finger[3][0] < finger[2][0] and finger[2][0] < finger[1][0] and finger[1][0] < finger[0][0]):
        return True
    return False

In [8]:
def isBetween(left, middle, right):
    if (middle[0] > min(left[0], right[0]) and middle[0] < max(left[0], right[0])):
        return True
    return False

In [9]:
def isBetweenVertical(left, middle, right):
    if (middle[1] > min(left[1], right[1]) and middle[1] < max(left[1], right[1])):
        return True
    return False

In [10]:
def faceFront(indexKnuckle, wrist, pinkyKnuckle):
    if (calculate_angle(indexKnuckle, wrist, pinkyKnuckle) > 30):
        return True
    return False

In [11]:
def isParallel(line1, line2):
    angle1 = calculate_angle(line1[0], line1[1], line2[0])
    angle2 = calculate_angle(line1[0], line2[0], line2[0])
    if (abs(angle1-angle2) < 8):
        return True
    else:
        return False

In [12]:
def displayText(text):
    cv2.putText(image, text, 
                                      (int(width*0.5), 300), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 7, (255, 255, 255), 2, cv2.LINE_AA
                                            )

In [13]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands


# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
        if cap.isOpened():
            width  = cap.get(3)  # float `width`
            height = cap.get(4)
            print (str(width) + " " + str(height))
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                  print("Ignoring empty camera frame.")
                  # If loading a video, use 'break' instead of 'continue'.
                  continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.        
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            results = hands.process(image)
            
            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)            
            
            try:
                landmarks = results.hand_landmarks.landmark
                print(landmarks)
            except:
                pass

    


            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
                    
                  
                
                #finger landmarks-----------------------
                #18 - pinky second knuckle, 10-middle second knuckle, 4-thumb tip
                
                #numbered starting from knuckle
                #index[1][0] --> x value of first joint down
                index = []
                middle = []
                ring = []
                thumb = []
                pinky = []
                for mark in range(0,4):
                    thumb.append([hand_landmarks.landmark[mark+1].x, hand_landmarks.landmark[mark+1].y, hand_landmarks.landmark[mark+1].z])
                    index.append([hand_landmarks.landmark[mark+5].x, hand_landmarks.landmark[mark+5].y, hand_landmarks.landmark[mark+5].z])
                    middle.append([hand_landmarks.landmark[mark+9].x, hand_landmarks.landmark[mark+9].y, hand_landmarks.landmark[mark+9].z])
                    ring.append([hand_landmarks.landmark[mark+13].x, hand_landmarks.landmark[mark+13].y, hand_landmarks.landmark[mark+13].z])
                    pinky.append([hand_landmarks.landmark[mark+17].x, hand_landmarks.landmark[mark+17].y, hand_landmarks.landmark[mark+17].z])
                
                #chekcs if facing front
                #front = faceFront(index[0], [hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y], pinky[0])
                #slope = getAbsSlope(index[0], pinky[0]) #slope of line between pinky + index knuckle. higher value --> side facing
                
            
                #checking for actions-------------------------
                #ok (index + thumb tip distance r close)
              
                if max(calculate_distance(pinky[3], ring[3]), calculate_distance(pinky[3], middle[3]), calculate_distance(pinky[3], index[3])) < 0.05 and pinky[3][1] > middle[1][1]:   
                        if calculate_distance(pinky[3], thumb[3]) < 0.05:
                            displayText("o")
                        else:
                            displayText("c")
                
                elif (bentFinger(index) and bentFinger(middle) and bentFinger(ring) and bentFinger(pinky)):
                    if (isBetween(middle[1], index[1], thumb[3]) and thumb[3][1] < middle[1][1]):
                        displayText("a")
                    elif (isBetween(middle[1], thumb[3], index[1]) and thumb[3][1] < middle[1][1]):
                        displayText("t")
                    elif(isBetween(ring[1], thumb[3], middle[1]) and thumb[3][1] < ring[1][1]):
                        displayText("n")
                    elif(isBetween(pinky[1], thumb[3], ring[1]) and thumb[3][1] < pinky[1][1]):
                        displayText("m")
                    elif(isBetween(pinky[1], thumb[3], index[1]) and thumb[3][1] < middle[2][1] and thumb[3][1] > middle[1][1]):
                        displayText("s")
                    elif(isBetween(pinky[1], thumb[3], index[1]) and thumb[3][1] > middle[2][1]):
                        displayText("e")

                elif(straightFinger(index) and straightFinger(middle) and straightFinger(ring) and straightFinger(pinky)):
                    if (isBetween(pinky[0], thumb[3], index[0])):
                        displayText("b")
                elif(bentFinger(index) and straightFinger(middle) and straightFinger(ring) and straightFinger(pinky)):
                    if (calculate_distance(thumb[3], index[3]) < 0.1):
                        displayText("f")
                elif(not bentFinger(index) and bentFinger(middle) and bentFinger(ring) and bentFinger(pinky) and pinky[3][1] > pinky[2][1]):
                    if(index[3][1] > index[1][1] and index[3][1] < middle[1][1]):
                        displayText("x")
                    elif (calculate_distance(thumb[3], ring[3]) < 0.08):
                        displayText("d")
                    elif(straightFinger(index) and calculate_angle(index[3], thumb[0], thumb[3]) > 30):
                        displayText("l")
                elif(straightFinger(index) and straightFinger(middle) and bentFinger(ring) and bentFinger(pinky)):
                    if (index[3][0] > middle[3][0]):
                        displayText("r")  
                    else:
        
                        if (calculate_angle(index[3], middle[0], middle[3]) > 18):
                            if isBetween(middle[1], thumb[3], index[1]) and thumb[3][1] < middle[0][1]:
                                displayText("k")
                            else:
                                displayText("v")
                        else:
                            displayText("u")
                elif(not bentFinger(index) and not bentFinger(middle) and not bentFinger(ring) and bentFinger(pinky)):
                    if (calculate_angle(index[3], middle[0], ring[3]) > 30):
                         displayText("w")
                elif(bentFinger(index) and bentFinger(middle) and bentFinger(ring) and straightFinger(pinky)):
                    if (calculate_angle(pinky[3], middle[3], thumb[3]) < 20):
                         displayText("i")
                    elif (calculate_angle(pinky[3], middle[3],thumb[3]) > 45 and thumb[3][1] < index[1][1]):
                         displayText("y")
                elif isLeft(index) and isLeft(middle) and straightDown(middle) and isBetweenVertical(index[2], thumb[3], middle[2]) and ring[3][0] > ring[1][0] and pinky[3][0] > pinky[1][0]:
                    displayText("p")
                elif isLeft(index) and isLeft(middle) and ring[3][0] > ring[1][0] and pinky[3][0] > pinky[1][0] and calculate_distance(thumb[3], ring[1]) < 0.5:
                    displayText("h")
                elif isLeft(index) and isLeft(thumb) and middle[3][0] > middle[1][0] and ring[3][0] > ring[1][0] and pinky[3][0] > pinky[1][0]:
                    if straightDown(index):
                        displayText("q")
                    else:
                        displayText("g")
                                       
                
                
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Hands', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cap.release
cv2.destroyAllWindows()

640.0 480.0


In [14]:
print(calculate_distance)

<function calculate_distance at 0x0000024F6CE30B80>


In [12]:
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
print(arr[0:-1:2])

[1, 3, 5, 7, 9]


In [ ]:
                        
                '''     
                elif (maxTip - minTip)< 0.03 and slope < 0.3: #algined fingertips (along y axis)
                    
                    if (maxTip > middle[1][1]+0.05): # if finger is bending down     (tip of finger is below middle knuckle)
                        if (thumb[2][2] > index[2][2]): #index joint in front of thumb joint
                            if (thumb[2][2] > middle[2][2]): #middle joint in front of thumb joint
                                if (thumb[2][2] > ring[2][2]):
                                    displayText("m")
                                else:
                                    displayText("n")
                            else:
                                displayText("t")
                        elif (calculate_angle(pinky[1], middle[1], thumb[3]) > 150 ): # a, thumb at side
                             displayText("a")
                        elif(calculate_angle(pinky[1], middle[1], thumb[3]) < 100 ): #  e or s, thumb in middle
                            if(thumb[3][1] > middle[2][1]): #thumb below fingers (e)
                                    displayText("e")
                            elif(thumb[3][1] < middle[2][1] and thumb[3][1] > middle[1][1]): #thumb ontop of fingers (s)
                                    displayText("s")
                '''